# Analytické funkce v pandas

Analytické funkce umožňují zpracovávat data bez agregace - můžeme extrahovat informace o skupině, aniž bychom ztratili informace o jednotlivých prvcích.

**Základní analytické metody:**
- `cumcount()` - číslo řádku (pořadí prvku ve skupině)
- `cumsum()` - kumulativní součet
- `cummin()` - kumulativní minimum
- `cummax()` - kumulativní maximum

**Postup použití:**
1. Seřadíme DataFrame podle požadovaného sloupce
2. Seskupíme data a aplikujeme analytickou funkci

## Načtení a příprava dat

In [ ]:
import pandas as pd

In [ ]:
# Načtení dat
df = pd.read_csv(
    r'..\01_data\processed\product_prices_cleaned.csv',
    sep=';'
)

In [ ]:
# Kontrola načtení dat
df.head()

In [ ]:
# Filtrování dat - vybereme pouze jeden produkt a region
df = df.loc[
    (df['product'] == 'barley groats sausage - per 1kg') &
    (df['province'] == 'POLAND')
]

In [ ]:
# Kontrola filtrování
df.head()

In [ ]:
# Převod sloupce date na datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

---
## 1. Číslo řádku - cumcount()

Funkce `cumcount()` určuje pořadí prvku ve skupině. Používá se např. pro vytvoření rankingu.

In [ ]:
# Seřazení podle hodnoty (sestupně - nejvyšší ceny první)
df_sorted = df.sort_values(by=['value'], ascending=False)

In [ ]:
# Přidání čísla řádku pomocí cumcount()
# Pozor: cumcount() není implementován pro DataFrame, pouze pro GroupBy objekt
df_sorted['rn'] = df_sorted.groupby(by=['product']).cumcount()

In [ ]:
# Kontrola - cumcount() vrací hodnoty od 0 do n-1
df_sorted.head()

In [ ]:
# Výběr top 10 měsíců s nejvyšší cenou
df_sorted.query("rn < 10")

### Otázka k zamyšlení

Proč `cumcount()` vrací hodnoty od 0 a ne od 1? Jak byste získali ranking začínající od 1?

---
## 2. Kumulativní součet - cumsum()

Funkce `cumsum()` počítá průběžný součet hodnot.

In [ ]:
# Seřazení podle hodnoty
df_sorted = df_sorted.sort_values(by=['value'], ascending=False)

In [ ]:
# Přidání kumulativního součtu
# Pozor: musíme specifikovat sloupec 'value', jinak by se sečetly všechny číselné sloupce
df_sorted['cs'] = df_sorted.groupby(by=['product'])['value'].cumsum()

In [ ]:
df_sorted.head()

### Úloha - oprav chybu

Následující kód obsahuje chybu. Najdi ji a oprav.

In [ ]:
# Chybný kód - oprav ho
df_sorted['cumsum_wrong'] = df_sorted['value'].cumsum()

---
## 3. Kumulativní minimum - cummin()

Funkce `cummin()` sleduje, jak se vyvíjí minimum v čase.

In [ ]:
# Seřazení podle data (chronologicky)
df_sorted = df_sorted.sort_values(by=['date'], ascending=True)

In [ ]:
# Přidání kumulativního minima
df_sorted['cummin'] = df_sorted.groupby(by=['product'])['value'].cummin()

In [ ]:
df_sorted.head()

---
## 4. Kumulativní maximum - cummax()

Funkce `cummax()` sleduje, jak se vyvíjí maximum v čase.

In [ ]:
# Seřazení podle data
df_sorted = df_sorted.sort_values(by=['date'], ascending=True)

In [ ]:
# Přidání kumulativního maxima
df_sorted['cummax'] = df_sorted.groupby(by=['product'])['value'].cummax()

In [ ]:
df_sorted.head()

---
## Úlohy k procvičení

### Úloha 1 - Doplň kód

Vytvoř ranking produktů podle ceny (od nejnižší po nejvyšší). Doplň chybějící části kódu.

In [ ]:
# Seřaď data vzestupně podle sloupce 'value'
df_uloha = df.sort_values(by=___, ascending=___)

# Přidej sloupec 'rank' s pořadím
df_uloha['rank'] = df_uloha.groupby(by=['product']).___

### Úloha 2 - Oprav chybu

Následující kód by měl spočítat kumulativní maximum, ale obsahuje chyby. Oprav je.

In [ ]:
# Chybný kód
df_chyba = df.sort_values(by=['date'])
df_chyba['max'] = df_chyba.cummax()  # Zde jsou 2 chyby

### Úloha 3 - Vytvoř celý kód

Vytvoř kód, který:
1. Seřadí data podle data (chronologicky)
2. Přidá sloupec `cummin` s kumulativním minimem ceny
3. Přidá sloupec `cummax` s kumulativním maximem ceny
4. Zobrazí prvních 10 řádků

In [ ]:
# Tvůj kód zde


### Úloha 4 - Bonus

Pomocí `cumcount()` a `cumsum()` vypočítej průběžný průměr ceny (kumulativní průměr).

Nápověda: průměr = součet / počet

In [ ]:
# Tvůj kód zde


---
## Shrnutí - použité metody a funkce

| Metoda/Funkce | Popis | Příklad použití |
|--------------|-------|----------------|
| `sort_values(by, ascending)` | Seřazení DataFrame podle sloupce | `df.sort_values(by=['value'], ascending=False)` |
| `groupby(by)` | Seskupení dat podle sloupce | `df.groupby(by=['product'])` |
| `cumcount()` | Pořadové číslo prvku ve skupině (0 až n-1) | `df.groupby('col').cumcount()` |
| `cumsum()` | Kumulativní (průběžný) součet | `df.groupby('col')['value'].cumsum()` |
| `cummin()` | Kumulativní minimum | `df.groupby('col')['value'].cummin()` |
| `cummax()` | Kumulativní maximum | `df.groupby('col')['value'].cummax()` |
| `query()` | Filtrování dat pomocí výrazu | `df.query("rn < 10")` |
| `pd.to_datetime()` | Převod na datetime | `pd.to_datetime(df['date'], format='%Y-%m')` |

**Důležité poznámky:**
- Analytické funkce (`cumcount`, `cumsum`, `cummin`, `cummax`) nejsou implementovány přímo pro DataFrame - musí se použít po `groupby()`
- Před použitím analytických funkcí je nutné data seřadit podle požadovaného kritéria
- `cumcount()` vrací hodnoty od 0 do n-1